In [1]:
%run "ZeroMQ Python Connector".py
%load_ext autoreload
%autoreload
import pandas as pd

In [2]:
#Create connector instance, with particular port for PUSH, PULL and SUB
connector = DWX_ZeroMQ_Connector()

[INIT] Ready to send commands to METATRADER (PUSH): 32768
[INIT] Listening for responses from METATRADER (PULL): 32769
[INIT] Listening for market data from METATRADER (SUB): 32770


In [14]:
ID = connector._DWX_MTX_SEND_MARKETDATA_REQUEST_(_symbol='GBPUSD', _start=(Timestamp.now()-Timedelta(days=2)).strftime('%Y.%m.%d %H:%M:00'),_timeframe=1440,_end=Timestamp.now().strftime('%Y.%m.%d %H:%M:00'))
response = connector._get_response_(messageID = ID)
print(response)

{'_messageID': 1734497286, '_action': 'DATA', '_data': {'2019.05.24 00:00': 1.27113}}
{'_messageID': 1734497286, '_action': 'DATA', '_data': {'2019.05.24 00:00': 1.27113}}


In [ ]:
response = Series(connector._get_response_())
responseData = Series(response["_data"])
print(type(responseData[0]))
order = connector._generate_default_order_dict()
print(order)

In [ ]:
order = connector._generate_default_order_dict()
connector._DWX_MTX_SEND_MARKETDATA_REQUEST_(_symbol='GBPUSD', _start=(Timestamp.now()-Timedelta(minutes=1)).strftime('%Y.%m.%d %H:%M:00'),_timeframe=1,_end=Timestamp.now().strftime('%Y.%m.%d %H:%M:00'))
response = Series(connector._get_response_())
responseData = Series(response["_data"])
order['_symbol'] = 'GBPUSD'
order['_price'] = responseData[0]
#connector._DWX_MTX_NEW_TRADE_(order)

In [ ]:
message = connector._DWX_MTX_GET_ALL_OPEN_TRADES_()
print(message)

In [ ]:
def obtainMarketDataChunk(symbol,startTime, endTime, timeframe=1):
    outputFrame = pd.DataFrame()
    
    if timeframe == 1:
        chunkSizeMinutes = 125
        chunkSizeDays = 0
    elif timeframe == 1440:
        chunkSizeDays = 125
        chunkSizeMinutes = 0
    pdStartTime = Timestamp.strptime(startTime,'%Y.%m.%d %H:%M:00')
    pdEndTime = Timestamp.strptime(endTime,'%Y.%m.%d %H:%M:00')
    timeDelta = pdEndTime - pdStartTime 
    divMod = divmod(timeDelta,Timedelta(minutes=chunkSizeMinutes, days = chunkSizeDays))
    currentStartTime = None
    currentEndTime = None
    response = None
    print(divMod)
    for x in range(divMod[0]):
        print("--------------------------------------------------------------------------------------------------")
        oldResponse = connector._get_response_()
        currentStartTime = pdStartTime + (x*Timedelta(minutes=chunkSizeMinutes, days = chunkSizeDays))
        currentEndTime = pdStartTime + ((x+1)*Timedelta(minutes=chunkSizeMinutes, days = chunkSizeDays))
        connector._DWX_MTX_SEND_MARKETDATA_REQUEST_(_symbol=symbol, _start=currentStartTime.strftime('%Y.%m.%d %H:%M:00'),
                                                    _timeframe=timeframe,_end=currentEndTime.strftime('%Y.%m.%d %H:%M:00'))
        while (connector._get_response_() == oldResponse):
            pass
        
        response = connector._get_response_()
        outputFrame = outputFrame.append(Series(response['_data']).to_frame())
        sleep(0.1)
        print("--------------------------------------------------------------------------------------------------")
        
    
    print("--------------------------------------------------------------------------------------------------")
    connector._DWX_MTX_SEND_MARKETDATA_REQUEST_(_symbol=symbol, _start=currentEndTime.strftime('%Y.%m.%d %H:%M:00'),
                                                _timeframe=timeframe,_end=endTime)
    oldResponse = connector._get_response_()
    while (connector._get_response_() == oldResponse):
            pass
    response = connector._get_response_()
    outputFrame = outputFrame.append(Series(response['_data']).to_frame())
    print("--------------------------------------------------------------------------------------------------")
    return outputFrame.reset_index()


In [ ]:
datachunk = obtainMarketDataChunk('GBPUSD', '2018.10.22 10:00:00', '2019.05.23 01:00:00',timeframe=1440)
print(datachunk.to_string())